<a href="https://colab.research.google.com/github/ValLovaton/rag_wikipedia-lab/blob/valeria-task2/notebooks/rag_wikipedia.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install wikipedia-api sentence-transformers chromadb langchain transformers torch pandas


  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 3.2 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.8/20.8 MB 36.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 278.2/278.2 kB 13.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 42.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.3/103.3 kB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.4/17.4 MB 46.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.5/72.5 kB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.3/132.3 kB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.9/65.9 kB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 208.0/208.0 kB 9.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [3]:
import wikipediaapi
import pandas as pd
import re

# 1. Descargar artículo
wiki = wikipediaapi.Wikipedia(
    user_agent="rag-wikipedia-lab (valeria & sebas)",
    language="en"
)

page = wiki.page("Federated_learning")

# 2. Extraer texto principal
text = page.text

# 3. Dividir en chunks de ~300 palabras
def chunk_text(text, chunk_size=300):
    words = text.split()
    chunks = []
    for i in range(0, len(words), chunk_size):
        chunk = " ".join(words[i:i+chunk_size])
        chunks.append(chunk)
    return chunks

chunks = chunk_text(text, 300)

# 4. Crear DataFrame para guardar como CSV
df = pd.DataFrame({
    "id": list(range(len(chunks))),
    "title": ["Federated_learning"] * len(chunks),
    "text": chunks
})

df.head()


,id,title,text
0,0,Federated_learning,Federated learning (also known as collaborativ...
1,1,Federated_learning,smartphones and IoT devices) compared to distr...
2,2,Federated_learning,federated learning In the decentralized federa...
3,3,Federated_learning,"according to the server inputs, a machine lear..."
4,4,Federated_learning,and identically distributed samples across loc...


In [4]:
# Guardar el corpus en el repositorio local del notebook
csv_path = "/content/wiki_corpus.csv"
df.to_csv(csv_path, index=False)

csv_path


'/content/wiki_corpus.csv'

In [5]:
# 1. Cargar el CSV
import pandas as pd

df = pd.read_csv("/content/wiki_corpus.csv")
df.head()


,id,title,text
0,0,Federated_learning,Federated learning (also known as collaborativ...
1,1,Federated_learning,smartphones and IoT devices) compared to distr...
2,2,Federated_learning,federated learning In the decentralized federa...
3,3,Federated_learning,"according to the server inputs, a machine lear..."
4,4,Federated_learning,and identically distributed samples across loc...


In [6]:
from sentence_transformers import SentenceTransformer
import chromadb

# 2. Cargar modelo de embeddings
model = SentenceTransformer("all-MiniLM-L6-v2")

# 3. Crear cliente y colección
client = chromadb.Client()

collection = client.create_collection(
    name="wiki_ai",
    metadata={"hnsw:space": "cosine"}
)

# 4. Insertar chunks en ChromaDB
ids = df["id"].astype(str).tolist()
metadatas = [{"title": t} for t in df["title"]]
documents = df["text"].tolist()

# Generar embeddings
embeddings = model.encode(documents).tolist()

# Upsert
collection.upsert(
    ids=ids,
    embeddings=embeddings,
    documents=documents,
    metadatas=metadatas
)

"Vector store creado con éxito"


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

'Vector store creado con éxito'